<a href="https://colab.research.google.com/github/manish284/Natural-Language-Processing/blob/master/Hand%20Written%20Text%20Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os 
import urllib.request

Downloading The required data from [IAM DataSet](http://www.fki.inf.unibe.ch/databases/iam-handwriting-database/download-the-iam-handwriting-database)

In [ ]:
def getIAMData(folderName,url,username,password):
  cwd = os.getcwd()
  if not os.path.exists(os.path.join(cwd, folderName)):
    os.mkdir(os.path.join(cwd, folderName))
    os.chdir(os.path.join(cwd, folderName))
  else:
    os.chdir(os.path.join(cwd, folderName))
  password_mgr = urllib.request.HTTPPasswordMgrWithDefaultRealm()
  password_mgr.add_password(None, url, username, password)
  handler = urllib.request.HTTPBasicAuthHandler(password_mgr)
  opener = urllib.request.build_opener(handler)
  urllib.request.install_opener(opener)
  opener.open(url)
  filename = os.path.basename(url)
  print("Downloading {}: ".format(filename))
  urllib.request.urlretrieve(url, filename=os.path.join(os.getcwd(), filename))[0]


In [ ]:
url= 'http://www.fki.inf.unibe.ch/DBs/iamDB/data/ascii/ascii.tgz'
username='####'
password = '####'
getIAMData('data', url, username, password)
!tar -xvzf /content/data/ascii.tgz

./
forms.txt
lines.txt
words.txt
sentences.txt


In [ ]:
url= 'http://www.fki.inf.unibe.ch/DBs/iamDB/data/words/words.tgz'
getIAMData('words', url, username, password)
!tar -xvzf /content/data/words/words.tgz
os.chdir('/content')

Streaming output truncated to the last 5000 lines.
r06/r06-137/r06-137-05-09.png
r06/r06-137/r06-137-05-10.png
r06/r06-137/r06-137-05-11.png
r06/r06-137/r06-137-06-00.png
r06/r06-137/r06-137-06-01.png
r06/r06-137/r06-137-06-02.png
r06/r06-137/r06-137-06-03.png
r06/r06-137/r06-137-06-04.png
r06/r06-137/r06-137-06-05.png
r06/r06-137/r06-137-06-06.png
r06/r06-137/r06-137-06-07.png
r06/r06-137/r06-137-06-08.png
r06/r06-137/r06-137-07-00.png
r06/r06-137/r06-137-07-01.png
r06/r06-137/r06-137-07-02.png
r06/r06-137/r06-137-07-03.png
r06/r06-137/r06-137-07-04.png
r06/r06-137/r06-137-07-05.png
r06/r06-137/r06-137-07-06.png
r06/r06-137/r06-137-07-07.png
r06/r06-137/r06-137-07-08.png
r06/r06-137/r06-137-07-09.png
r06/r06-137/r06-137-07-10.png
r06/r06-137/r06-137-07-11.png
r06/r06-137/r06-137-07-12.png
r06/r06-137/r06-137-08-00.png
r06/r06-137/r06-137-08-01.png
r06/r06-137/r06-137-08-02.png
r06/r06-137/r06-137-08-03.png
r06/r06-137/r06-137-08-04.png
r06/r06-137/r06-137-08-05.png
r06/r06-137/r06-137

In [ ]:
os.getcwd()

'/content'

Data preprocessing

In [ ]:
import numpy as np
import cv2
from keras.preprocessing.sequence import pad_sequences

In [ ]:
char_list = "!\"#&'()*+,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz" 

In [ ]:
def encode_to_labels(txt):
  dig_lst = []
  for index, chara in enumerate(txt):
    dig_lst.append(char_list.index(chara))
  return dig_lst

In [ ]:
def process_image(img):

  w, h = img.shape
  new_w = 32
  new_h = int(h * (new_w / w))
  img = cv2.resize(img, (new_h, new_w))
  w, h = img.shape
  img = img.astype('float32')

  if w < 32:
    add_zeros = np.full((32-w, h), 255)
    img = np.concatenate((img, add_zeros))
    w, h = img.shape
    
  if h < 128:
    add_zeros = np.full((w, 128-h), 255)
    img = np.concatenate((img, add_zeros), axis=1)
    w, h = img.shape
        
  if h > 128 or w > 32:
    dim = (128,32)
    img = cv2.resize(img, dim)
    
  img = cv2.subtract(255, img) 
  img = np.expand_dims(img, axis=2)  
  img = img / 255
  return img

In [ ]:
def ReadWordFile():
  with open('/content/data/words.txt') as f:
    contents = f.readlines()
  lines = [line.strip() for index, line in enumerate(contents) if index >=18]
  np.random.shuffle(lines)
  return lines[0:15000]

In [ ]:
def DataProcessing(char_list):
  max_label_len = 0
  images  = []
  labels  = []
  input_length = []
  label_length = []
  original_text = []
  lines = ReadWordFile()
  for index, line in enumerate(lines):
    splits = line.split(' ')
    status = splits[1]
    if status == 'ok':
      word_id = splits[0]
      word = "".join(splits[8:])
        
      splits_id = word_id.split('-')
      filepath = '/content/data/words/{}/{}-{}/{}.png'.format(splits_id[0], splits_id[0], splits_id[1], word_id)
        
      img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
      try:
        img = process_image(img)
      except:
        continue
      try:
        label = encode_to_labels(word)
      except:
        continue
      
      images.append(img)
      labels.append(label)
      input_length.append(31)
      label_length.append(len(word))
      #original_text.append(word)

      if len(word) > max_label_len:
        max_label_len = len(word)

  padded_label = pad_sequences(labels, maxlen = max_label_len, padding='post', value=len(char_list))

  images = np.asarray(images)
  input_length = np.asarray(input_length)
  label_length = np.asarray(label_length)

  return images, padded_label, input_length, label_length, max_label_len

In [ ]:
import os
import cv2
import string
import numpy as np
import pandas as pd
from keras.models import Model
from keras.layers import Dense, LSTM, Reshape, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional
from keras.activations import relu, sigmoid, softmax
import keras.backend as K
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

In [ ]:
def baseModel(inputs,char_list):
  conv_1 = Conv2D(64, (3,3), activation = 'relu', padding='same')(inputs)
  pool_1 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_1)
  conv_2 = Conv2D(128, (3,3), activation = 'relu', padding='same')(pool_1)
  pool_2 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_2)
  conv_3 = Conv2D(256, (3,3), activation = 'relu', padding='same')(pool_2) 
  conv_4 = Conv2D(256, (3,3), activation = 'relu', padding='same')(conv_3)
  pool_4 = MaxPool2D(pool_size=(2, 1))(conv_4) 
  conv_5 = Conv2D(512, (3,3), activation = 'relu', padding='same')(pool_4)
  batch_norm_5 = BatchNormalization()(conv_5)
  conv_6 = Conv2D(512, (3,3), activation = 'relu', padding='same')(batch_norm_5)
  batch_norm_6 = BatchNormalization()(conv_6)
  pool_6 = MaxPool2D(pool_size=(2, 1))(batch_norm_6)
  conv_7 = Conv2D(512, (2,2), activation = 'relu')(pool_6)
  squeezed = Lambda(lambda x: K.squeeze(x, 1))(conv_7)
  blstm_1 = Bidirectional(LSTM(256, return_sequences=True, dropout = 0.2))(squeezed)
  blstm_2 = Bidirectional(LSTM(256, return_sequences=True, dropout = 0.2))(blstm_1)
  outputs = Dense(len(char_list)+1, activation = 'softmax')(blstm_2)
  model   = Model(inputs, outputs)
  return model, outputs

In [ ]:
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

In [ ]:
def createModel(max_label_len):
  inputs = Input(shape=(32,128,1))
  the_labels    = Input(name='the_labels', shape=[max_label_len], dtype='float32')
  input_length  = Input(name='input_length', shape=[1], dtype='int64')
  label_length  = Input(name='label_length', shape=[1], dtype='int64')
  bModel, outputs = baseModel(inputs,char_list)
  loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([outputs, the_labels, input_length, label_length])
  model  = Model(inputs=[inputs, the_labels, input_length, label_length], outputs=loss_out)
  return model

In [ ]:
def trainModel():
  data_split =0.2
  batch_size = 8
  epochs = 30
  e = str(epochs)
  optimizer_name = 'sgd'
  images, padded_label, input_length, label_length,max_label_len = DataProcessing(char_list)
  val_index   = int(len(images) *data_split)
  train_index = len(images) - val_index
  model = createModel(max_label_len)
  model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer = optimizer_name, metrics=['accuracy'])
  if not os.path.exists(os.path.join(os.getcwd(), 'Model')):
    os.mkdir(os.path.join(os.getcwd(), 'Model'))
  filepath="/content/Model/{}o-{}r-{}e-{}t-{}v.hdf5".format(optimizer_name, str(images.shape(0)), str(epochs), str(images[0:train_index].shape(0)), str(images[train_index:].shape(0)))
  checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
  callbacks_list = [checkpoint]
  history = model.fit(x=[images[0:train_index], padded_label[0:train_index], input_length[0:train_index], label_length[0:train_index]],
                    y=np.zeros(len(images[0:train_index])),
                    batch_size=batch_size, 
                    epochs=epochs, 
                    validation_data=([images[train_index:], padded_label[train_index:], input_length[train_index:], label_length[train_index:]], [np.zeros(len(images[train_index:]))]),
                    verbose=2,
                    callbacks=callbacks_list)

In [ ]:
trainModel()

In [ ]:
!pip install python-levenshtein

In [ ]:
act_model.load_weights(filepath)

In [ ]:
prediction = act_model.predict(valid_images)

In [ ]:
decoded = K.ctc_decode(prediction, 
                       input_length=np.ones(prediction.shape[0]) * prediction.shape[1],
                       greedy=True)[0][0]
out = K.get_value(decoded)

In [ ]:
import Levenshtein as lv

In [ ]:
total_jaro = 0
total_rati = 0
# see the results
for i, x in enumerate(out):
    letters=''
    for p in x:
        if int(p) != -1:
            letters+=char_list[int(p)]
    total_jaro+=lv.jaro(letters, valid_original_text[i])
    total_rati+=lv.ratio(letters, valid_original_text[i])

print('jaro :', total_jaro/len(out))
print('ratio:', total_rati/len(out))

In [ ]:

# predict outputs on validation images
prediction =act_model.predict(train_images[542:645])
 
# use CTC decoder
decoded = K.ctc_decode(prediction,   
                       input_length=np.ones(prediction.shape[0]) * prediction.shape[1],
                       greedy=True)[0][0]

out = K.get_value(decoded)

# see the results
for i, x in enumerate(out):
    print("original_text =  ", train_original_text[542+i])
    print("predicted text = ", end = '')
    for p in x:
        if int(p) != -1:
            print(char_list[int(p)], end = '')
    plt.imshow(train_images[542+i].reshape(32,128), cmap=plt.cm.gray)
    plt.show()
    print('\n')

In [ ]:

def plotgraph(epochs, acc, val_acc):
    # Plot training & validation accuracy values
    plt.plot(epochs, acc, 'b')
    plt.plot(epochs, val_acc, 'r')
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper left')
    plt.show()

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1,len(loss)+1)

In [ ]:

plotgraph(epochs, loss, val_loss)

In [ ]:
plotgraph(epochs, acc, val_acc)

In [ ]:
minimum_val_loss = np.min(history.history['val_loss'])
best_model_index = np.where(history.history['val_loss'] == minimum_val_loss)[0][0]

best_loss = str(history.history['loss'][best_model_index])
best_acc = str(history.history['accuracy'][best_model_index])
best_val_loss = str(history.history['val_loss'][best_model_index])
best_val_acc = str(history.history['val_accuracy'][best_model_index])